In [ ]:
# Chatbot using Retrieval Augmented Generation
# Uses Groq Inference Platform to run LLM online

# TODO: benötigt groq API Key in GROQ_API_KEY
# TODO: erstelle Ordner pdfs und lade ein oder mehrere PDFs in diesen Ordner hoch

# install packages
!apt-get install -y poppler-utils tesseract-ocr libmagic1
!pip install "unstructured[all-docs]" python-magic llama-index==0.12.35 llama-index-vector-stores-chroma llama-index-embeddings-huggingface chromadb pdf2image pytesseract gradio plotly

!pip install git+https://github.com/dgaida/llm_client.git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
libmagic1 is already the newest version (1:5.41-3ubuntu0.1).
libmagic1 set to manually installed.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (265 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126281 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# --- Import required libraries ---
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext
from llama_index.readers.file.unstructured import UnstructuredReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Settings
from chromadb.config import Settings as ChromaSettings
import gradio as gr
import os
import chromadb
import random
from llm_client import LLMClient

# --- Step 1: Set up PDF file path ---
# Put your PDF files in a folder named "pdfs" in the current directory
PDF_DIR = "pdfs"

In [2]:
# --- Step 2: Load and parse PDFs with uMiner (via UnstructuredReader) ---
# --- Check if PDF directory exists ---
if not os.path.exists(PDF_DIR):
    raise FileNotFoundError(f"The folder '{PDF_DIR}' does not exist. Please create it and add PDF files.")

reader = UnstructuredReader()
all_documents = []

# Loop through PDF files in the folder and read each one individually
for filename in os.listdir(PDF_DIR):
    print("File:" + filename)
    if filename.lower().endswith(".pdf"):
        file_path = os.path.join(PDF_DIR, filename)
        docs = reader.load_data(file_path)
        all_documents.extend(docs)

print(f"Loaded {len(all_documents)} documents")
for doc in all_documents[:3]:
    print(doc.text[:300])  # Zeige ersten Ausschnitt

File:17 Good enough practices in scientific computing.pdf


Loaded 1 documents
PERSPECTIVE

Good enough practices in scientific computing

Greg Wilson1☯*, Jennifer Bryan2☯, Karen Cranston3☯, Justin Kitzes4☯, Lex Nederbragt5☯, Tracy K. Teal6☯

1 Software Carpentry Foundation,Austin, Texas, United States of America, 2 RStudio and Department of Statistics, University of British C


In [3]:
# Optional: split documents into smaller chunks for better retrieval
node_parser = SentenceSplitter(chunk_size=256, chunk_overlap=0)  # 512, 50

In [4]:
# this is the embedding model
# models can be found here: https://huggingface.co/spaces/mteb/leaderboard
embed_model = HuggingFaceEmbedding(model_name = "intfloat/e5-small-v2")
#embed_model = HuggingFaceEmbedding(model_name = "sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
client = LLMClient()

In [ ]:
from llama_index.core.llms import LLM
from llama_index.core.llms import ChatMessage, CompletionResponse, ChatResponse
from typing import List, Optional
from pydantic import Field
from llama_index.core.llms import LLMMetadata

# class that we need so that we can use the LLM provided by Hugging Face
# inference and pass it to llama_index as a "normal" LLM
class HuggingFaceLLM(LLM):
    client: Optional[LLMClient] = Field(default=None, exclude=True)

    def __init__(self, **data):
        super().__init__(**data)

    def chat(self, messages: List[ChatMessage], **kwargs) -> ChatResponse:
        hf_messages = [
            {"role": m.role, "content": m.content} for m in messages
        ]

        response = self.client.chat_completion(hf_messages)

        print(response)

        return ChatResponse(
            message=ChatMessage(role="assistant", content=response)
        )

    def complete(self, prompt: str, **kwargs) -> CompletionResponse:
        raise NotImplementedError("complete not implemented")

    def stream_chat(self, *args, **kwargs):
        raise NotImplementedError("stream_chat not implemented")

    def stream_complete(self, *args, **kwargs):
        raise NotImplementedError("stream_complete not implemented")

    async def astream_chat(self, *args, **kwargs):
        raise NotImplementedError("astream_chat not implemented")

    async def astream_complete(self, *args, **kwargs):
        raise NotImplementedError("astream_complete not implemented")

    async def achat(self, *args, **kwargs):
        raise NotImplementedError("achat not implemented")

    async def acomplete(self, *args, **kwargs):
        raise NotImplementedError("acomplete not implemented")

    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=2048,
            num_output=512,
            is_chat_model=True,
            model_name=self.model
        )

In [ ]:
# this is the LLM used to answer our questions
llm = HuggingFaceLLM(client=client)

# Apply settings globally - LlamaIndex uses this Settings object internally
Settings.llm = llm  # None
Settings.embed_model = embed_model
Settings.node_parser = node_parser

In [12]:
# --- Step 4: Create vector store and index ---
# Chroma will store vectors in a temporary database
chroma_client = chromadb.EphemeralClient()

# Get the current collections (if any) and clear them
existing_collections = chroma_client.list_collections()
# Loop through the collections and delete them by name
for collection in existing_collections:
    # Extract the collection name and delete it
    collection_name = collection.name  # Assuming the 'Collection' object has a 'name' attribute
    chroma_client.delete_collection(collection_name)

# unfortunately, if you delete a collection, it still seems to be there. Those
# deletions also do not help. If you really want to delete a database you have
# to restart the session
if 'chroma_collection' in locals():
    print("Deleting existing chroma_collection")
    del chroma_collection

if 'vector_store' in locals():
    print("Deleting existing vector_store")
    del vector_store

if 'index' in locals():
    del index

# add random number because deleting a collection not really deletes it -
# this also does not help...
chroma_collection = chroma_client.create_collection(f"dlml{random.randint(0,100)}")

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

# here the documents are stored in the database
index = VectorStoreIndex.from_documents(
    all_documents,
    storage_context=storage_context
)

In [38]:
from huggingface_hub.utils import HfHubHTTPError

# --- Step 5: Create query engine ---
query_engine = index.as_query_engine()

In [39]:
# this function does everyting.
# takes the query of the user and embeds it into the vector space,
# does a semantic search in the vector database (compares vectors)
# returns the text of the most similar vector
# passes this text as context to the LLM together with the query
# retrieves the response of the LLM and returns it as a string
# llama_index does this all internally using the Settings object.
def chat_with_pdf(query, history=None):
    # Frage den Vektor-Index ab, der jetzt automatisch den HF LLM nutzt
    try:
      response = query_engine.query(query)
    except HfHubHTTPError as e:
      print(e)
      response = "error with Huggingface inference provider"

    return str(response)

In [40]:
print(chat_with_pdf("how to format my disk?"))

ChatCompletion(id='chatcmpl-b4718d28-5a3a-4e46-af5a-8d8bdd9af3e1', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='There is no information provided about formatting a disk. The text discusses data storage solutions, backup strategies, and the cost of storage, but does not mention disk formatting.', role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=None))], created=1752483207, model='llama-3.3-70b-versatile', object='chat.completion', system_fingerprint='fp_6507bcfb6f', usage=CompletionUsage(completion_tokens=34, prompt_tokens=606, total_tokens=640, completion_time=0.118235715, prompt_time=0.037327044, queue_time=0.200929335, total_time=0.155562759), usage_breakdown=None, x_groq={'id': 'req_01k04275t1eb7vss26hqjrqvwr'}, service_tier='on_demand')
There is no information provided about formatting a disk. The text discusses data storage solutions, backup strategies, and the cost of storage, but does no

In [41]:
# --- Step 7: Build Gradio Interface ---
# This creates a simple web UI for asking questions
chat_ui = gr.ChatInterface(
    fn=chat_with_pdf,
    title="PDF RAG Chatbot",
    description="Ask questions about the content of your PDF documents.",
    theme="default",
    examples=["What is this PDF about?", "Summarize the second section."],
)

chat_ui.launch()

/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8a89607919d1206a83.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
